In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time
import pandas as pd
from copy import deepcopy

# Compatibility fix for numpy
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

# ========== Hyperparameters ==========
gamma = 0.99
hidden_dim = 256
learning_rate = 1e-3
episodes = 1000
lambda_fixed = 20  # Lagrange multiplier
b = 200.0          # cost threshold buffer
perturb_eps = 0    # Uniform noise for state perturbation
delta = 0.05
# ========== Environment ==========
env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# ========== Neural Networks ==========

class Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)
        )
    def forward(self, state):
        return self.model(state)

class ValueCritic(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, state):
        return self.model(state)

# ========== Networks & Optimizers ==========
actor = Actor()
reward_critic = ValueCritic()
cost_critic = ValueCritic()

actor_optim = optim.Adam(actor.parameters(), lr=learning_rate)
reward_optim = optim.Adam(reward_critic.parameters(), lr=learning_rate)
cost_optim = optim.Adam(cost_critic.parameters(), lr=learning_rate)

# ========== Utility Functions ==========

def add_uniform_noise(state, eps=0.05):
    """Uniform perturbation across each dimension of state."""
    #noise = np.random.uniform(0, eps, size=state.shape)
    return state

def discount(values, gamma=0.99):
    result = []
    G = 0
    for v in reversed(values):
        G = v + gamma * G
        result.insert(0, G)
    return torch.FloatTensor(result)

# ========== Training Loop ==========

dataF = {'cost': [], 'reward': []}
last_50_actor_params = []  # To store actor weights for averaging
best_reward = float('-inf')
best_actor_state_dict = None

start = time.time()
for ep in range(episodes):
    state = env.reset()
    state = add_uniform_noise(np.array(state), perturb_eps)
    state = torch.FloatTensor(state)

    log_probs = []
    rewards = []
    costs = []
    reward_values = []
    cost_values = []

    total_reward = 0
    total_cost = 0
    done = False

    while not done:
        probs = actor(state)
        dist = Categorical(probs)
        action = dist.sample()

        next_state, reward, done, _ = env.step(action.item())

        next_state = add_uniform_noise(np.array(next_state), perturb_eps)
        next_state = torch.FloatTensor(next_state)

        cost = abs(state[0].item())  # distance-based cost

        # Save transitions
        log_probs.append(dist.log_prob(action))
        rewards.append(reward)
        costs.append(cost)
        reward_values.append(reward_critic(state))
        cost_values.append(cost_critic(state))

        total_reward += reward
        total_cost += cost
        state = next_state

    # Discounted returns
    reward_returns = discount(rewards, gamma)
    cost_returns = discount(costs, gamma)

    reward_values = torch.cat(reward_values).squeeze()
    cost_values = torch.cat(cost_values).squeeze()
    log_probs = torch.stack(log_probs)

    adv_r = reward_returns - reward_values.detach()
    adv_c = cost_returns - cost_values.detach()

    chosen_adv = []
    for vr, vc, ar, ac in zip(reward_returns, cost_returns, adv_r, adv_c):
        if vc.item()<=b+delta:
            chosen_adv.append(ar)
        else:
            chosen_adv.append(-ac)  # penalize constraint
    chosen_adv = torch.stack(chosen_adv)

    # ===== Losses =====
    actor_loss = -(log_probs * chosen_adv).mean()
    reward_loss = nn.functional.mse_loss(reward_values, reward_returns)
    cost_loss = nn.functional.mse_loss(cost_values, cost_returns)

    # ===== Backprop =====
    actor_optim.zero_grad()
    actor_loss.backward()
    actor_optim.step()

    reward_optim.zero_grad()
    reward_loss.backward()
    reward_optim.step()

    cost_optim.zero_grad()
    cost_loss.backward()
    cost_optim.step()

    dataF['cost'].append(total_cost)
    dataF['reward'].append(total_reward)

    # === Store weights for last 50 actor policies ===
    if len(last_50_actor_params) >= 50:
        last_50_actor_params.pop(0)
    last_50_actor_params.append(deepcopy(actor.state_dict()))

    if total_cost < b and total_reward > best_reward:
        best_reward = total_reward
        best_actor_state_dict = deepcopy(actor.state_dict())

    # Print progress
    if (ep + 1) % 50 == 0:
        print(f"Ep {ep+1} | Reward: {total_reward:.1f} | Cost: {total_cost:.2f} | Actor Loss: {actor_loss.item():.3f}")

# ========== After Training ==========

# Save data
env.close()
df = pd.DataFrame(dataF)
df.to_excel('tvf_and_tcf_data_crpo.xlsx')

# Save final models
torch.save(actor.state_dict(), 'actor_crpo.pth')
torch.save(reward_critic.state_dict(), 'reward_critic_crpo.pth')
torch.save(cost_critic.state_dict(), 'cost_critic_crpo.pth')

# === Average Last 50 Actor Policies ===
avg_actor_state_dict = deepcopy(last_50_actor_params[0])
for key in avg_actor_state_dict.keys():
    for i in range(1, len(last_50_actor_params)):
        avg_actor_state_dict[key] += last_50_actor_params[i][key]
    avg_actor_state_dict[key] /= len(last_50_actor_params)

avg_actor = Actor()
avg_actor.load_state_dict(avg_actor_state_dict)
torch.save(avg_actor.state_dict(), 'actor_avg_last50_crpo.pth')

# Save best actor if available
if best_actor_state_dict is not None:
    best_actor = Actor()
    best_actor.load_state_dict(best_actor_state_dict)
    torch.save(best_actor.state_dict(), 'actor_best_crpo.pth')
print("Time taken", time.time()-start)
print("Training complete. Models saved.")

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Ep 50 | Reward: 65.0 | Cost: 2.30 | Actor Loss: 15.560
Ep 100 | Reward: 46.0 | Cost: 0.63 | Actor Loss: 9.226
Ep 150 | Reward: 141.0 | Cost: 123.05 | Actor Loss: 20.735
Ep 200 | Reward: 178.0 | Cost: 93.10 | Actor Loss: 18.476
Ep 250 | Reward: 154.0 | Cost: 51.07 | Actor Loss: 10.554
Ep 300 | Reward: 500.0 | Cost: 243.23 | Actor Loss: 22.044
Ep 350 | Reward: 181.0 | Cost: 98.64 | Actor Loss: 0.060
Ep 400 | Reward: 412.0 | Cost: 399.40 | Actor Loss: 10.790
Ep 450 | Reward: 388.0 | Cost: 310.05 | Actor Loss: 5.575
Ep 500 | Reward: 500.0 | Cost: 235.20 | Actor Loss: 8.389
Ep 550 | Reward: 500.0 | Cost: 355.78 | Actor Loss: 1.659
Ep 600 | Reward: 500.0 | Cost: 307.41 | Actor Loss: 1.480
Ep 650 | Reward: 500.0 | Cost: 209.70 | Actor Loss: 2.507
Ep 700 | Reward: 500.0 | Cost: 380.38 | Actor Loss: 2.634
Ep 750 | Reward: 500.0 | Cost: 192.33 | Actor Loss: 1.860
Ep 800 | Reward: 500.0 | Cost: 283.12 | Actor Loss: -0.025
Ep 850 | Reward: 500.0 | Cost: 101.84 | Actor Loss: 1.009
Ep 900 | Reward: 